In [ ]:
!pip install -U transformers datasets trl peft accelerate bitsandbytes

In [ ]:
## important imports
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
import datasets
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv("Barbie_Movie_Dataset.csv")
df.head()

In [ ]:
df["formatted_instruction"] = df.apply(lambda x: f"### Instruction:\n{x['text']}\n\n### Response:\n{x['labels']}", axis=1)

In [ ]:
df.head()

In [ ]:
from huggingface_hub import login
login(token="")

In [ ]:
!nvidia-smi

In [ ]:
base_model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

In [ ]:
df["formatted_instruction_tok_len"] = df["formatted_instruction"].apply(lambda x:len(tokenizer.encode(x)))


In [ ]:
df = df[df["formatted_instruction_tok_len"]<=128]
df.shape

In [ ]:
dataset = datasets.Dataset.from_pandas(df)

In [ ]:
dataset

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)


In [ ]:
import time

In [ ]:
start = time.time()
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1
print("time for model load: {} seconds".format(time.time()-start))

## Before fine tuning

In [ ]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
prompt = "### Instruction:\nTell me about the movie Barbie\n\n### Response:\n"
gen_text = pipe(prompt)
print(gen_text[0]['generated_text'][len(prompt):])

In [ ]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
prompt = "### Instruction:\nPlease tell me in brief about the cast of the movie called Barbie\n\n### Response:\n"
# prompt = "###Instruction:\nPlease tell me in brief about the cast of the movie called \"Barbie\"\n\n###Response\n:"
gen_text = pipe(prompt)
print(gen_text[0]['generated_text'][len(prompt):])

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="formatted_instruction",
    max_seq_length=128,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

In [ ]:
start = time.time()
output = trainer.train()
print("Time taken: ", time.time()-start)

In [ ]:
## After

## After

In [ ]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
prompt = "###Instruction:\nPlease tell me in brief about the movie called \"Barbie\"\n\n###Response\n:"
start = time.time()
gen_text = pipe(prompt)
print(gen_text[0]['generated_text'][len(prompt):])
print("Time taken: ", time.time()-start)